In [1]:
from typing import List, Dict, Optional, Tuple
import re
import unicodedata

class BuildingNameNormalizer:
    def __init__(self):
        # 建物名の一般的な表記パターン
        self.building_patterns = {
            r'ビル(?:ディング)?': 'ビル',
            r'マンション': 'マンション',
            r'コーポ(?:ラス)?': 'コーポ',
            r'ハイツ': 'ハイツ',
            r'メゾン': 'メゾン',
            r'アパート': 'アパート',
            r'パレス': 'パレス',
            r'ハウス': 'ハウス',
            r'レジデンス': 'レジデンス',
            r'タワー(?:ズ)?': 'タワー',
        }

        # 建物名の接尾語パターン
        self.suffix_patterns = {
            r'１(?:st|ST)|一': '1',
            r'２(?:nd|ND)|二': '2',
            r'３(?:rd|RD)|三': '3',
            r'４(?:th|TH)|四': '4',
            r'５(?:th|TH)|五': '5',
            r'号(?:館|棟)?': '号館',
            r'アネックス': 'ANNEX',
            r'別館': '別館',
            r'新館': '新館',
            r'本館': '本館',
        }

        # 建物名の組織パターン
        self.org_patterns = {
            r'株式会社|㈱': '(株)',
            r'有限会社|㈲': '(有)',
            r'財団法人': '(財)',
            r'社団法人': '(社)',
        }

        # 方角を表す接頭語
        self.direction_patterns = {
            r'東(?:側)?': '東',
            r'西(?:側)?': '西',
            r'南(?:側)?': '南',
            r'北(?:側)?': '北',
        }

    def normalize_building_name(self, building_name: str) -> Tuple[str, dict]:
        """
        建物名を正規化し、変換情報を返す

        Args:
            building_name (str): 正規化したい建物名

        Returns:
            Tuple[str, dict]: (正規化された建物名, 変換情報の辞書)
        """
        # 変換履歴を記録する辞書
        conversion_info = {
            'original': building_name,
            'steps': []
        }

        # 空白文字の正規化
        normalized = unicodedata.normalize('NFKC', building_name)
        normalized = re.sub(r'\s+', ' ', normalized).strip()
        if normalized != building_name:
            conversion_info['steps'].append(('空白文字の正規化', normalized))

        # カタカナの正規化（半角→全角）
        pre_kana = normalized
        normalized = self._normalize_katakana(normalized)
        if normalized != pre_kana:
            conversion_info['steps'].append(('カタカナの正規化', normalized))

        # 建物種別の正規化
        for pattern, replacement in self.building_patterns.items():
            pre_building = normalized
            normalized = re.sub(pattern, replacement, normalized, flags=re.IGNORECASE)
            if normalized != pre_building:
                conversion_info['steps'].append((f'建物種別の正規化: {pattern}→{replacement}', normalized))

        # 接尾語の正規化
        for pattern, replacement in self.suffix_patterns.items():
            pre_suffix = normalized
            normalized = re.sub(pattern, replacement, normalized, flags=re.IGNORECASE)
            if normalized != pre_suffix:
                conversion_info['steps'].append((f'接尾語の正規化: {pattern}→{replacement}', normalized))

        # 組織名の正規化
        for pattern, replacement in self.org_patterns.items():
            pre_org = normalized
            normalized = re.sub(pattern, replacement, normalized)
            if normalized != pre_org:
                conversion_info['steps'].append((f'組織名の正規化: {pattern}→{replacement}', normalized))

        # 方角の正規化
        for pattern, replacement in self.direction_patterns.items():
            pre_direction = normalized
            normalized = re.sub(pattern, replacement, normalized)
            if normalized != pre_direction:
                conversion_info['steps'].append((f'方角の正規化: {pattern}→{replacement}', normalized))

        conversion_info['final'] = normalized
        return normalized, conversion_info

    def _normalize_katakana(self, text: str) -> str:
        """
        カタカナを正規化する（半角→全角）
        """
        return unicodedata.normalize('NFKC', text)

    def compare_building_names(self, name1: str, name2: str) -> Tuple[float, List[str]]:
        """
        2つの建物名の類似度を計算し、差分情報を返す

        Args:
            name1 (str): 比較する建物名1
            name2 (str): 比較する建物名2

        Returns:
            Tuple[float, List[str]]: (類似度スコア, 差分情報のリスト)
        """
        # 両方の建物名を正規化
        norm1, info1 = self.normalize_building_name(name1)
        norm2, info2 = self.normalize_building_name(name2)

        # 文字列の編集距離を計算
        distance = self._levenshtein_distance(norm1, norm2)
        max_length = max(len(norm1), len(norm2))
        similarity = 1 - (distance / max_length)

        # 差分情報の収集
        differences = []
        if norm1 != norm2:
            differences.append(f"正規化後の表記が異なります:")
            differences.append(f"  建物名1: {norm1}")
            differences.append(f"  建物名2: {norm2}")

        return similarity, differences

    def _levenshtein_distance(self, s1: str, s2: str) -> int:
        """
        レーベンシュタイン距離を計算する
        """
        if len(s1) < len(s2):
            return self._levenshtein_distance(s2, s1)

        if len(s2) == 0:
            return len(s1)

        previous_row = range(len(s2) + 1)
        for i, c1 in enumerate(s1):
            current_row = [i + 1]
            for j, c2 in enumerate(s2):
                insertions = previous_row[j + 1] + 1
                deletions = current_row[j] + 1
                substitutions = previous_row[j] + (c1 != c2)
                current_row.append(min(insertions, deletions, substitutions))
            previous_row = current_row

        return previous_row[-1]

# 使用例
if __name__ == "__main__":
    normalizer = BuildingNameNormalizer()
    
    # テストケース
    test_cases = [
        # 一般的な建物名の表記揺れ
        {
            "inputs": [
                "住友不動産新宿グランドタワー",
                "住友不動産 新宿グランドタワービル",
                "新宿グランドタワービルディング",
                "新宿グランドタワー"
            ],
            "expected": "新宿グランドタワー"
        },
        # マンション名の表記揺れ
        {
            "inputs": [
                "グリーンハイツ１号館",
                "グリーンハイツ 1号棟",
                "グリーンハイツ一号",
                "グリーンハイツ1st"
            ],
            "expected": "グリーンハイツ1号館"
        },
        # 組織名を含む建物名
        {
            "inputs": [
                "株式会社東京建物本社ビル",
                "㈱東京建物本社ビルディング",
                "東京建物(株)ビル",
                "東京建物ビル本館"
            ],
            "expected": "(株)東京建物ビル本館"
        },
        # 方角を含む建物名
        {
            "inputs": [
                "横浜駅東口ビル",
                "横浜駅東側ビルディング",
                "横浜駅東ビル",
                "ﾖｺﾊﾏｴｷヒガシビル"  # 半角カタカナの例
            ],
            "expected": "横浜駅東ビル"
        }
    ]
    
    # テストの実行
    for i, test_case in enumerate(test_cases, 1):
        print(f"\nテストケース {i}")
        print("=" * 50)
        
        for building_name in test_case["inputs"]:
            normalized, info = normalizer.normalize_building_name(building_name)
            print(f"\n入力建物名: {building_name}")
            print(f"正規化結果: {normalized}")
            print(f"期待される結果: {test_case['expected']}")
            
            print("\n変換ステップ:")
            for step, result in info['steps']:
                print(f"- {step}")
                print(f"  → {result}")
            
            # 期待値との類似度を計算
            similarity, differences = normalizer.compare_building_names(
                normalized, test_case["expected"]
            )
            print(f"\n期待値との類似度: {similarity:.2f}")
            if differences:
                print("差分情報:")
                for diff in differences:
                    print(f"  {diff}")
            
            print("-" * 50)


テストケース 1

入力建物名: 住友不動産新宿グランドタワー
正規化結果: 住友不動産新宿グランドタワー
期待される結果: 新宿グランドタワー

変換ステップ:

期待値との類似度: 0.64
差分情報:
  正規化後の表記が異なります:
    建物名1: 住友不動産新宿グランドタワー
    建物名2: 新宿グランドタワー
--------------------------------------------------

入力建物名: 住友不動産 新宿グランドタワービル
正規化結果: 住友不動産 新宿グランドタワービル
期待される結果: 新宿グランドタワー

変換ステップ:

期待値との類似度: 0.53
差分情報:
  正規化後の表記が異なります:
    建物名1: 住友不動産 新宿グランドタワービル
    建物名2: 新宿グランドタワー
--------------------------------------------------

入力建物名: 新宿グランドタワービルディング
正規化結果: 新宿グランドタワービル
期待される結果: 新宿グランドタワー

変換ステップ:
- 建物種別の正規化: ビル(?:ディング)?→ビル
  → 新宿グランドタワービル

期待値との類似度: 0.82
差分情報:
  正規化後の表記が異なります:
    建物名1: 新宿グランドタワービル
    建物名2: 新宿グランドタワー
--------------------------------------------------

入力建物名: 新宿グランドタワー
正規化結果: 新宿グランドタワー
期待される結果: 新宿グランドタワー

変換ステップ:

期待値との類似度: 1.00
--------------------------------------------------

テストケース 2

入力建物名: グリーンハイツ１号館
正規化結果: グリーンハイツ1号館
期待される結果: グリーンハイツ1号館

変換ステップ:
- 空白文字の正規化
  → グリーンハイツ1号館

期待値との類似度: 1.00
--------------------------------------------------

入力建物名: グリーンハイツ 1号棟
